# Nuclear Plant Simulation with Constant Heat Source

This notebook demonstrates running the nuclear plant simulator with a constant heat source instead of complex reactor physics. This is perfect for:

- Testing secondary side systems
- Development and debugging
- Understanding plant dynamics without reactor complexity
- Educational purposes

## Key Features

✅ **Instant Response**: Heat source responds immediately to power commands  
✅ **Predictable Behavior**: No complex reactor physics or dynamics  
✅ **Easy Control**: Simple power setpoint control  
✅ **Full Plant Model**: Complete thermal hydraulics and steam cycle  

## Setup and Imports

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Dict, Any
import time
from datetime import datetime

# Add project root to path
sys.path.append('.')

# Import nuclear simulator components
from core.sim import NuclearPlantSimulator, ControlAction
from primary_system.core.heat_sources import ConstantHeatSource
from utils.plant_data_logger import PlantDataLogger
from utils.plant_plotter import PlantPlotter

# Configure matplotlib for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ All imports successful!")
print(f"📅 Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Create Constant Heat Source Simulator

Let's create a nuclear plant simulator using a constant heat source instead of reactor physics.

In [ ]:
# Create constant heat source
heat_source = ConstantHeatSource(rated_power_mw=3000.0)

# Create simulator with constant heat source
simulator = NuclearPlantSimulator(dt=1.0, heat_source=heat_source)

print("🔥 Heat Source Configuration:")
print(f"   Type: {heat_source.__class__.__name__}")
print(f"   Rated Power: {heat_source.rated_power_mw:.0f} MW")
print(f"   Current Power: {heat_source.get_power_percent():.1f}%")
print(f"   Efficiency: {heat_source.get_efficiency():.1%}")

print("\n🏭 Simulator Configuration:")
print(f"   Time Step: {simulator.dt:.1f} seconds")
print(f"   Initial Power Level: {simulator.state.power_level:.1f}%")
print(f"   Initial Fuel Temperature: {simulator.state.fuel_temperature:.1f}°C")
print(f"   Initial Coolant Temperature: {simulator.state.coolant_temperature:.1f}°C")
print(f"   Initial Control Rod Position: {simulator.state.control_rod_position:.1f}%")

## Basic Simulation Run

Let's run a basic simulation to see how the constant heat source behaves.

In [ ]:
def run_basic_simulation(duration_seconds: int = 300) -> pd.DataFrame:
    """
    Run a basic simulation and return results as DataFrame
    """
    # Reset simulator
    simulator.reset()
    
    # Data collection
    data = []
    
    print(f"🚀 Running basic simulation for {duration_seconds} seconds...")
    print(f"{'Time (s)':<8} {'Power (%)':<10} {'Fuel T (°C)':<12} {'Coolant T (°C)':<14} {'Status':<10}")
    print("-" * 70)
    
    for t in range(duration_seconds):
        # Step simulation
        result = simulator.step()
        
        # Collect data
        data.append({
            'time': simulator.time,
            'power_level': simulator.state.power_level,
            'fuel_temperature': simulator.state.fuel_temperature,
            'coolant_temperature': simulator.state.coolant_temperature,
            'coolant_pressure': simulator.state.coolant_pressure,
            'control_rod_position': simulator.state.control_rod_position,
            'steam_flow_rate': simulator.state.steam_flow_rate,
            'steam_pressure': simulator.state.steam_pressure,
            'thermal_power_mw': result['info']['thermal_power'],
            'scram_status': simulator.state.scram_status
        })
        
        # Print status every 60 seconds
        if t % 60 == 0:
            status = "SCRAM" if simulator.state.scram_status else "Normal"
            print(f"{simulator.time:<8.0f} {simulator.state.power_level:<10.1f} "
                  f"{simulator.state.fuel_temperature:<12.0f} "
                  f"{simulator.state.coolant_temperature:<14.1f} {status:<10}")
        
        # Check for early termination
        if result['done']:
            print(f"\n⚠️  Simulation terminated early at {simulator.time:.0f}s due to safety system activation")
            break
    
    print(f"\n✅ Simulation completed!")
    print(f"   Final time: {simulator.time:.0f} seconds")
    print(f"   Final power: {simulator.state.power_level:.1f}%")
    print(f"   Final fuel temperature: {simulator.state.fuel_temperature:.1f}°C")
    
    return pd.DataFrame(data)

# Run the simulation
basic_data = run_basic_simulation(300)

## Plot Basic Results

In [ ]:
def plot_simulation_results(data: pd.DataFrame, title: str = "Nuclear Plant Simulation Results"):
    """
    Plot simulation results in a comprehensive dashboard
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    # Power Level
    axes[0, 0].plot(data['time'], data['power_level'], 'b-', linewidth=2, label='Power Level')
    axes[0, 0].axhline(y=100, color='g', linestyle='--', alpha=0.7, label='Rated Power')
    axes[0, 0].set_ylabel('Power Level (%)')
    axes[0, 0].set_title('Power Level')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Temperatures
    axes[0, 1].plot(data['time'], data['fuel_temperature'], 'r-', linewidth=2, label='Fuel Temperature')
    axes[0, 1].plot(data['time'], data['coolant_temperature'], 'b-', linewidth=2, label='Coolant Temperature')
    axes[0, 1].set_ylabel('Temperature (°C)')
    axes[0, 1].set_title('Temperatures')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # Pressure
    axes[0, 2].plot(data['time'], data['coolant_pressure'], 'g-', linewidth=2, label='Coolant Pressure')
    axes[0, 2].plot(data['time'], data['steam_pressure'], 'orange', linewidth=2, label='Steam Pressure')
    axes[0, 2].set_ylabel('Pressure (MPa)')
    axes[0, 2].set_title('System Pressures')
    axes[0, 2].grid(True, alpha=0.3)
    axes[0, 2].legend()
    
    # Control Rod Position
    axes[1, 0].plot(data['time'], data['control_rod_position'], 'purple', linewidth=2)
    axes[1, 0].set_ylabel('Rod Position (%)')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_title('Control Rod Position')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Steam Flow Rate
    axes[1, 1].plot(data['time'], data['steam_flow_rate'], 'cyan', linewidth=2)
    axes[1, 1].set_ylabel('Steam Flow (kg/s)')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_title('Steam Flow Rate')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Thermal Power
    axes[1, 2].plot(data['time'], data['thermal_power_mw'], 'red', linewidth=2)
    axes[1, 2].axhline(y=3000, color='g', linestyle='--', alpha=0.7, label='Rated Power')
    axes[1, 2].set_ylabel('Thermal Power (MW)')
    axes[1, 2].set_xlabel('Time (s)')
    axes[1, 2].set_title('Thermal Power Output')
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].legend()
    
    plt.tight_layout()
    plt.show()

# Plot the basic results
plot_simulation_results(basic_data, "Basic Constant Heat Source Simulation")

## Power Control Demonstration

Now let's demonstrate the key advantage of the constant heat source: instant response to power commands.

In [ ]:
def run_power_control_demo(duration_seconds: int = 600) -> pd.DataFrame:
    """
    Demonstrate power control capabilities with constant heat source
    """
    # Reset simulator
    simulator.reset()
    
    # Data collection
    data = []
    
    print(f"🎛️  Running power control demonstration for {duration_seconds} seconds...")
    print("\nPower Control Schedule:")
    print("  0-120s:   100% power (baseline)")
    print("  120-180s: 80% power (load reduction)")
    print("  180-240s: 110% power (load increase)")
    print("  240-300s: 60% power (minimum load)")
    print("  300-360s: 120% power (peak load)")
    print("  360-420s: 90% power (normal operation)")
    print("  420-480s: Power ramp 90% → 105%")
    print("  480-540s: Power ramp 105% → 75%")
    print("  540-600s: Return to 100%")
    print()
    
    for t in range(duration_seconds):
        # Define power control schedule
        if t < 120:
            target_power = 100.0  # Baseline
        elif t < 180:
            target_power = 80.0   # Load reduction
        elif t < 240:
            target_power = 110.0  # Load increase
        elif t < 300:
            target_power = 60.0   # Minimum load
        elif t < 360:
            target_power = 120.0  # Peak load
        elif t < 420:
            target_power = 90.0   # Normal operation
        elif t < 480:
            # Ramp from 90% to 105% over 60 seconds
            progress = (t - 420) / 60.0
            target_power = 90.0 + (105.0 - 90.0) * progress
        elif t < 540:
            # Ramp from 105% to 75% over 60 seconds
            progress = (t - 480) / 60.0
            target_power = 105.0 + (75.0 - 105.0) * progress
        else:
            # Return to 100%
            progress = (t - 540) / 60.0
            target_power = 75.0 + (100.0 - 75.0) * progress
        
        # Set heat source power (instant response!)
        simulator.heat_source.set_power_setpoint(target_power)
        
        # Step simulation
        result = simulator.step()
        
        # Collect data
        data.append({
            'time': simulator.time,
            'target_power': target_power,
            'actual_power': simulator.state.power_level,
            'fuel_temperature': simulator.state.fuel_temperature,
            'coolant_temperature': simulator.state.coolant_temperature,
            'coolant_pressure': simulator.state.coolant_pressure,
            'control_rod_position': simulator.state.control_rod_position,
            'steam_flow_rate': simulator.state.steam_flow_rate,
            'thermal_power_mw': result['info']['thermal_power'],
            'scram_status': simulator.state.scram_status
        })
        
        # Print status at power changes
        if t in [0, 120, 180, 240, 300, 360, 420, 480, 540] or t % 120 == 0:
            print(f"t={t:3d}s: Target={target_power:5.1f}%, Actual={simulator.state.power_level:5.1f}%, "
                  f"Fuel T={simulator.state.fuel_temperature:5.1f}°C")
        
        # Check for early termination
        if result['done']:
            print(f"\n⚠️  Simulation terminated early at {simulator.time:.0f}s due to safety system activation")
            break
    
    print(f"\n✅ Power control demonstration completed!")
    
    return pd.DataFrame(data)

# Run the power control demo
power_control_data = run_power_control_demo(600)

## Plot Power Control Results

In [ ]:
def plot_power_control_results(data: pd.DataFrame):
    """
    Plot power control demonstration results
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Constant Heat Source - Power Control Demonstration', fontsize=16, fontweight='bold')
    
    # Power tracking
    axes[0, 0].plot(data['time'], data['target_power'], 'r--', linewidth=2, label='Target Power', alpha=0.8)
    axes[0, 0].plot(data['time'], data['actual_power'], 'b-', linewidth=2, label='Actual Power')
    axes[0, 0].set_ylabel('Power Level (%)')
    axes[0, 0].set_title('Power Control Response')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    axes[0, 0].set_ylim(50, 130)
    
    # Power error
    power_error = data['actual_power'] - data['target_power']
    axes[0, 1].plot(data['time'], power_error, 'g-', linewidth=2)
    axes[0, 1].axhline(y=0, color='k', linestyle='-', alpha=0.5)
    axes[0, 1].set_ylabel('Power Error (%)')
    axes[0, 1].set_title('Power Control Error (Actual - Target)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Temperature response
    axes[1, 0].plot(data['time'], data['fuel_temperature'], 'r-', linewidth=2, label='Fuel Temperature')
    axes[1, 0].plot(data['time'], data['coolant_temperature'], 'b-', linewidth=2, label='Coolant Temperature')
    axes[1, 0].set_ylabel('Temperature (°C)')
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_title('Temperature Response to Power Changes')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    # Thermal power output
    axes[1, 1].plot(data['time'], data['thermal_power_mw'], 'purple', linewidth=2)
    axes[1, 1].axhline(y=3000, color='g', linestyle='--', alpha=0.7, label='Rated Power (3000 MW)')
    axes[1, 1].set_ylabel('Thermal Power (MW)')
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_title('Thermal Power Output')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display performance metrics
    print("\n📊 Power Control Performance Metrics:")
    print(f"   Mean Power Error: {power_error.mean():.3f}%")
    print(f"   Max Power Error: {power_error.abs().max():.3f}%")
    print(f"   RMS Power Error: {np.sqrt((power_error**2).mean()):.3f}%")
    print(f"   Power Range: {data['actual_power'].min():.1f}% - {data['actual_power'].max():.1f}%")
    print(f"   Temperature Range: {data['fuel_temperature'].min():.1f}°C - {data['fuel_temperature'].max():.1f}°C")

# Plot the power control results
plot_power_control_results(power_control_data)

## Load Following Scenario

Let's simulate a realistic load following scenario where the plant must respond to changing electrical demand.

In [ ]:
def run_load_following_scenario(duration_seconds: int = 1800) -> pd.DataFrame:
    """
    Simulate a realistic daily load following scenario
    """
    # Reset simulator
    simulator.reset()
    
    # Data collection
    data = []
    
    print(f"📈 Running load following scenario for {duration_seconds} seconds ({duration_seconds/60:.1f} minutes)...")
    print("\nScenario: Simulated daily demand curve (compressed to 30 minutes)")
    print("  - Morning ramp-up (6 AM equivalent)")
    print("  - Peak demand period (12 PM equivalent)")
    print("  - Afternoon variations")
    print("  - Evening peak (6 PM equivalent)")
    print("  - Night-time reduction")
    print()
    
    for t in range(duration_seconds):
        # Create realistic daily demand curve
        # Normalize time to 0-1 for a full "day" cycle
        time_normalized = (t / duration_seconds) * 2 * np.pi
        
        # Base demand with daily variation
        base_demand = 85.0  # Base load
        daily_variation = 15.0 * np.sin(time_normalized - np.pi/2)  # Daily cycle
        peak_boost = 8.0 * np.sin(2 * time_normalized) * (np.sin(time_normalized) > 0)  # Peak periods
        
        # Add some random variations (weather, industrial load, etc.)
        random_variation = 3.0 * np.sin(5 * time_normalized + 1.2) + 2.0 * np.sin(7 * time_normalized)
        
        # Calculate target power
        target_power = base_demand + daily_variation + peak_boost + random_variation
        target_power = np.clip(target_power, 60.0, 115.0)  # Realistic operating range
        
        # Set heat source power
        simulator.heat_source.set_power_setpoint(target_power)
        
        # Step simulation
        result = simulator.step()
        
        # Collect data
        data.append({
            'time': simulator.time,
            'time_hours': (t / duration_seconds) * 24,  # Convert to "hours" for plotting
            'target_power': target_power,
            'actual_power': simulator.state.power_level,
            'fuel_temperature': simulator.state.fuel_temperature,
            'coolant_temperature': simulator.state.coolant_temperature,
            'coolant_pressure': simulator.state.coolant_pressure,
            'steam_flow_rate': simulator.state.steam_flow_rate,
            'thermal_power_mw': result['info']['thermal_power'],
            'electrical_power_mw': result['info']['thermal_power'] * 0.33,  # Assume 33% efficiency
            'scram_status': simulator.state.scram_status
        })
        
        # Print status every 5 minutes (300 seconds)
        if t % 300 == 0:
            hour = (t / duration_seconds) * 24
            print(f"Hour {hour:4.1f}: Target={target_power:5.1f}%, Actual={simulator.state.power_level:5.1f}%, "
                  f"Electrical={result['info']['thermal_power'] * 0.33:6.1f} MW")
        
        # Check for early termination
        if result['done']:
            print(f"\n⚠️  Simulation terminated early at {simulator.time:.0f}s due to safety system activation")
            break
    
    print(f"\n✅ Load following scenario completed!")
    
    return pd.DataFrame(data)

# Run the load following scenario
load_following_data = run_load_following_scenario(1800)

## Plot Load Following Results

In [ ]:
def plot_load_following_results(data: pd.DataFrame):
    """
    Plot load following scenario results
    """
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    fig.suptitle('Load Following Scenario - Daily Demand Simulation', fontsize=16, fontweight='bold')
    
    # Daily demand curve
    axes[0, 0].plot(data['time_hours'], data['target_power'], 'r--', linewidth=2, label='Target Power', alpha=0.8)
    axes[0, 0].plot(data['time_hours'], data['actual_power'], 'b-', linewidth=2, label='Actual Power')
    axes[0, 0].set_ylabel('Power Level (%)')
    axes[0, 0].set_title('Daily Load Following')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    axes[0, 0].set_xlim(0, 24)
    
    # Electrical output
    axes[0, 1].plot(data['time_hours'], data['electrical_power_mw'], 'green', linewidth=2)
    axes[0, 1].axhline(y=1000, color='g', linestyle='--', alpha=0.7, label='Rated Electrical (1000 MW)')
    axes[0, 1].set_ylabel('Electrical Power (MW)')
    axes[0, 1].set_title('Electrical Power Output')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    axes[0, 1].set_xlim(0, 24)
    
    # Temperature response
    axes[1, 0].plot(data['time_hours'], data['fuel_temperature'], 'r-', linewidth=2, label='Fuel Temperature')
    axes[1, 0].plot(data['time_hours'], data['coolant_temperature'], 'b-', linewidth=2, label='Coolant Temperature')
    axes[1, 0].set_ylabel('Temperature (°C)')
    axes[1, 0].set_title('Temperature Response')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    axes[1, 0].set_xlim(0, 24)
    
    # Steam system response
    axes[1, 1].plot(data['time_hours'], data['steam_flow_rate'], 'cyan', linewidth=2, label='Steam Flow')
    axes[1, 1].plot(data['time_hours'], data['coolant_pressure'], 'orange', linewidth=2, label='Coolant Pressure')
    axes[1, 1].set_ylabel('Flow (kg/s) / Pressure (MPa)')
    axes[1, 1].set_title('Steam System Response')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].legend()
    axes[1, 1].set_xlim(0, 24)
    
    # Power tracking error
    power_error = data['actual_power'] - data['target_power']
    axes[2, 0].plot(data['time_hours'], power_error, 'purple', linewidth=2)
    axes[2, 0].axhline(y=0, color='k', linestyle='-', alpha=0.5)
    axes[2, 0].set_ylabel('Power Error (%)')
    axes[2, 0].set_xlabel('Time (hours)')
    axes[2, 0].set_title('Load Following Error')
    axes[2, 0].grid(True, alpha=0.3)
    axes[2, 0].set_xlim(0, 24)
    
    # Thermal power
    axes[2, 1].plot(data['time_hours'], data['thermal_power_mw'], 'red', linewidth=2)
    axes[2, 1].axhline(y=3000, color='g', linestyle='--', alpha=0.7, label='Rated Thermal (3000 MW)')
    axes[2, 1].set_ylabel('Thermal Power (MW)')
    axes[2, 1].set_xlabel('Time (hours)')
    axes[2, 1].set_title('Thermal Power Output')
    axes[2, 1].grid(True, alpha=0.3)
    axes[2, 1].legend()
    axes[2, 1].set_xlim(0, 24)
    
    plt.tight_layout()
    plt.show()
    
    # Performance metrics
    print("\n📊 Load Following Performance Metrics:")
    print(f"   Mean Power Error: {power_error.mean():.3f}%")
    print(f"   Max Power Error: {power_error.abs().max():.3f}%")
    print(f"   RMS Power Error: {np.sqrt((power_error**2).mean()):.3f}%")
    print(f"   Power Range: {data['actual_power'].min():.1f}% - {data['actual_power'].max():.1f}%")
    print(f"   Electrical Range: {data['electrical_power_mw'].min():.1f} - {data['electrical_power_mw'].max():.1f} MW")
    print(f"   Temperature Swing: {data['fuel_temperature'].max() - data['fuel_temperature'].min():.1f}°C")

# Plot the load following results
plot_load_following_results(load_following_data)

## Comparison with Reactor Physics

Let's create a comparison to show the difference between constant heat source and reactor physics.

In [ ]:
def compare_heat_sources(duration_seconds: int = 300) -> Dict[str, pd.DataFrame]:
    """
    Compare constant heat source vs reactor physics heat source
    """
    print("🔬 Comparing Heat Sources: Constant vs Reactor Physics")
    print("=" * 60)
    
    results = {}
    
    # Test with constant heat source
    print("\n1️⃣  Testing Constant Heat Source...")
    constant_heat_source = ConstantHeatSource(rated_power_mw=3000.0)
    constant_sim = NuclearPlantSimulator(dt=1.0, heat_source=constant_heat_source)
    constant_sim.reset()
    
    constant_data = []
    for t in range(duration_seconds):
        # Power step at t=60s
        if t == 60:
            constant_sim.heat_source.set_power_setpoint(80.0)
        elif t == 180:
            constant_sim.heat_source.set_power_setpoint(110.0)
        
        result = constant_sim.step()
        constant_data.append({
            'time': constant_sim.time,
            'power_level': constant_sim.state.power_level,
            'fuel_temperature': constant_sim.state.fuel_temperature,
            'heat_source_type': 'Constant'
        })
    
    results['constant'] = pd.DataFrame(constant_data)
    
    # Test with reactor physics heat source
    print("2️⃣  Testing Reactor Physics Heat Source...")
    from primary_system.core.heat_sources import ReactorHeatSource
    reactor_heat_source = ReactorHeatSource(rated_power_mw=3000.0)
    reactor_sim = NuclearPlantSimulator(dt=1.0, heat_source=reactor_heat_source)
    reactor_sim.reset()
    
    reactor_data = []
    for t in range(duration_seconds):
        # Control rod movements to change power
        if t == 60:
            action = ControlAction.CONTROL_ROD_INSERT
        elif t == 180:
            action = ControlAction.CONTROL_ROD_WITHDRAW
        else:
            action = None
        
        result = reactor_sim.step(action)
        reactor_data.append({
            'time': reactor_sim.time,
            'power_level': reactor_sim.state.power_level,
            'fuel_temperature': reactor_sim.state.fuel_temperature,
            'heat_source_type': 'Reactor Physics'
        })
    
    results['reactor'] = pd.DataFrame(reactor_data)
    
    print("✅ Comparison completed!")
    return results

# Run the comparison
comparison_data = compare_heat_sources(300)

In [ ]:
def plot_heat_source_comparison(data: Dict[str, pd.DataFrame]):
    """
    Plot comparison between heat sources
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Heat Source Comparison: Constant vs Reactor Physics', fontsize=16, fontweight='bold')
    
    # Power response comparison
    axes[0].plot(data['constant']['time'], data['constant']['power_level'], 
                'b-', linewidth=3, label='Constant Heat Source', alpha=0.8)
    axes[0].plot(data['reactor']['time'], data['reactor']['power_level'], 
                'r--', linewidth=2, label='Reactor Physics', alpha=0.8)
    axes[0].axvline(x=60, color='gray', linestyle=':', alpha=0.7, label='Control Actions')
    axes[0].axvline(x=180, color='gray', linestyle=':', alpha=0.7)
    axes[0].set_ylabel('Power Level (%)')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_title('Power Response Comparison')
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    # Temperature response comparison
    axes[1].plot(data['constant']['time'], data['constant']['fuel_temperature'], 
                'b-', linewidth=3, label='Constant Heat Source', alpha=0.8)
    axes[1].plot(data['reactor']['time'], data['reactor']['fuel_temperature'], 
                'r--', linewidth=2, label='Reactor Physics', alpha=0.8)
    axes[1].axvline(x=60, color='gray', linestyle=':', alpha=0.7, label='Control Actions')
    axes[1].axvline(x=180, color='gray', linestyle=':', alpha=0.7)
    axes[1].set_ylabel('Fuel Temperature (°C)')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_title('Temperature Response Comparison')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print("\n🔍 Heat Source Analysis:")
    print("\n📈 Constant Heat Source:")
    print("   ✅ Instant response to power commands")
    print("   ✅ Perfect power tracking")
    print("   ✅ Predictable behavior")
    print("   ✅ No complex physics to debug")
    print("   ⚠️  Not realistic for reactor training")
    
    print("\n⚛️  Reactor Physics Heat Source:")
    print("   ✅ Realistic reactor behavior")
    print("   ✅ Complex dynamics and feedback")
    print("   ✅ Training value for operators")
    print("   ⚠️  Slower response to control actions")
    print("   ⚠️  More complex to debug and tune")
    
    # Calculate response metrics
    const_power_change_60 = abs(data['constant']['power_level'].iloc[65] - data['constant']['power_level'].iloc[55])
    reactor_power_change_60 = abs(data['reactor']['power_level'].iloc[65] - data['reactor']['power_level'].iloc[55])
    
    print(f"\n📊 Response Speed (5s after control action):")
    print(f"   Constant Heat Source: {const_power_change_60:.1f}% power change")
    print(f"   Reactor Physics: {reactor_power_change_60:.1f}% power change")

# Plot the comparison
plot_heat_source_comparison(comparison_data)

## Summary and Conclusions

This notebook has demonstrated the nuclear plant simulator with a constant heat source.

In [ ]:
print("🎯 NUCLEAR PLANT CONSTANT HEAT SOURCE SIMULATION - SUMMARY")
print("=" * 70)
print()
print("✅ Successfully demonstrated:")
print("   • Basic plant simulation with constant heat source")
print("   • Instant power control response")
print("   • Complex load following scenarios")
print("   • Comparison with reactor physics")
print("   • Complete thermal hydraulics modeling")
print()
print("🔥 Constant Heat Source Benefits:")
print("   • Perfect for testing secondary systems")
print("   • Instant response to power commands")
print("   • Predictable and stable behavior")
print("   • Easy to debug and develop with")
print("   • No complex reactor physics to tune")
print()
print("🎓 Educational Value:")
print("   • Understand plant thermal hydraulics")
print("   • Learn control system behavior")
print("   • Study load following strategies")
print("   • Analyze secondary side dynamics")
print()
print("🚀 Next Steps:")
print("   • Try the reactor physics heat source for realism")
print("   • Experiment with different control strategies")
print("   • Add custom scenarios and disturbances")
print("   • Integrate with machine learning models")
print()
print(f"📅 Simulation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n🎉 Thank you for using the Nuclear Plant Simulator!")